In [1]:
import os
import pandas as pd
import time

import rasterio
import xarray as xr
import rioxarray as rioxr

import matplotlib.pyplot as plt

from scipy.ndimage import median_filter

In [21]:
year = 2020

scene_ids = pd.read_csv(os.path.join(os.getcwd(),
                                     'temp',
                                     str(year)+'_spectral_predictions',
                                     'coastal_scenes_ids_'+str(year)+'.csv')).itemid.iloc[19:30]
scene_ids

19    ca_m_3411936_sw_11_060_20200521
20    ca_m_3411936_se_11_060_20200521
21    ca_m_3411935_sw_11_060_20200521
22    ca_m_3411935_se_11_060_20200521
23    ca_m_3411934_sw_11_060_20200521
24    ca_m_3411934_se_11_060_20200521
25    ca_m_3411933_sw_11_060_20200521
26    ca_m_3411933_se_11_060_20200521
Name: itemid, dtype: object

In [22]:
times = []

for itemid in scene_ids:
    t0 = time.time()
    fp = os.path.join(os.getcwd(),
                  'temp',
                  str(year)+'_spectral_predictions',
                  'S_clip_preds_'+itemid+'.tif')
    
    raster = rioxr.open_rasterio(fp).squeeze()

    filtered = median_filter(raster.to_numpy(), size=3)
    
    dt = time.time()- t0
    times.append(dt)
    print('FINISHED: ', dt)
    
    filename = 'S_clip_filter_preds_'+itemid+'.tif'
    with rasterio.open(
            os.path.join(os.getcwd(),'temp',filename),  # file path
            'w',           # w = write
            driver = 'GTiff', # format
            height = raster.shape[0], 
            width = raster.shape[1],
            count = 1,  # number of raster bands in the dataset
            dtype = rasterio.uint8,
            crs = raster.rio.crs,
            transform = raster.rio.transform(),
        ) as dst:
            dst.write(filtered.astype(rasterio.uint8), 1)

FINISHED:  2.4245660305023193
FINISHED:  4.0341386795043945
FINISHED:  3.0659236907958984
FINISHED:  2.751966714859009
FINISHED:  2.6388983726501465
FINISHED:  2.9700019359588623
FINISHED:  0.8608381748199463
FINISHED:  3.6058428287506104


In [23]:
times

[2.4245660305023193,
 4.0341386795043945,
 3.0659236907958984,
 2.751966714859009,
 2.6388983726501465,
 2.9700019359588623,
 0.8608381748199463,
 3.6058428287506104]